In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [2]:
# import training dataset

X_train = pd.read_csv('../data/dengue_features_train.csv')
y_train = pd.read_csv('../data/dengue_labels_train.csv')

In [3]:
X_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [4]:
y_train.head(10)

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6
5,sj,1990,23,2
6,sj,1990,24,4
7,sj,1990,25,5
8,sj,1990,26,10
9,sj,1990,27,6


In [48]:
def preparing_dataset(X,y,column_list, weeks, split =False):
    '''
    I will write an abstract of this function :)
    -----
    Input:
        X (pd.DataFrame): X(input) dataset
        y (pd.DataFrame): y(label) dataset
        column_list (list): a list of columns you want to keep
        weeks (int):
            1: get environmental features one week ago
            2: get environmental features two weeks ago
            ...
        split (Boolean): (if a value is True) split the dataset by cities ('sj' or 'iq')
    -----
    Output:
        (if the value of split is False):
            df_X_train_concat (pd.DataFrame): 
            df_y_train_concat (pd.DataFrame): 
            df_X_val_concat (pd.DataFrame): 
            df_y_val_concat (pd.DataFrame): 
            
        (else):
            df_X_train_sj (pd.DataFrame): 
            df_y_train_sj (pd.DataFrame): 
            df_X_val_sj (pd.DataFrame): 
            df_y_val_sj (pd.DataFrame): 
            df_X_train_iq (pd.DataFrame): 
            df_y_train_iq (pd.DataFrame): 
            df_X_val_iq (pd.DataFrame): 
            df_y_val_iq (pd.DataFrame):    
    '''
    
    # make a copy of the X dataset
    df_X_train = X.copy()
    
   
    # fill missing values using pd.DataFrame interpolate 
    df_X_train = df_X_train.fillna(method = 'ffill')
    
    # change datatype of 'week_start_date' from string to np.datetime64
    df_X_train['week_start_date'] = pd.to_datetime(df_X_train['week_start_date'])
    
    # split the dataset by the city. 
    df_X_train_sj = df_X_train.loc[df_X_train['city']=='sj']
    df_X_train_iq = df_X_train.loc[df_X_train['city']=='iq']
    
    df_y_train_sj = y.loc[y['city']=='sj']
    df_y_train_iq = y.loc[y['city']=='iq']
    

    # this part is for transform the datasets to get environmental factors of three weeks ago 
    df_X_train_sj_time = df_X_train_sj.iloc[weeks:,:4].reset_index().iloc[:,1:]
    df_X_train_iq_time = df_X_train_iq.iloc[weeks:,:4].reset_index().iloc[:,1:]
    
    
    df_X_train_sj_features = df_X_train_sj.iloc[:-1*weeks,4:].reset_index().iloc[:,1:]
    df_X_train_iq_features = df_X_train_iq.iloc[:-1*weeks,4:].reset_index().iloc[:,1:]
    
    
    df_X_train_sj = pd.concat([df_X_train_sj_time,df_X_train_sj_features], axis=1)
    df_X_train_iq = pd.concat([df_X_train_iq_time,df_X_train_iq_features], axis=1) 
    
    df_X_train_sj =  df_X_train_sj.merge(y, left_on=['city','year','weekofyear'], right_on = ['city','year','weekofyear'], how= 'left')
    df_X_train_iq =  df_X_train_iq.merge(y, left_on=['city','year','weekofyear'], right_on = ['city','year','weekofyear'], how='left')
    
    # here I assigned 0 to city = sj and 1 to city = iq 
    df_X_train_sj['city'] = 0
    df_X_train_iq['city'] = 1    
    target_list = list(column_list)  
       
    # split train, validate dataset
    df_X_train_sj, df_X_val_sj = train_test_split(df_X_train_sj,train_size=0.7,shuffle=False)
    df_y_train_sj = df_X_train_sj['total_cases']
    df_X_train_sj = df_X_train_sj[target_list]
    df_y_val_sj = df_X_val_sj['total_cases']
    df_X_val_sj = df_X_val_sj[target_list]
        
    # split train, validate dataset
    df_X_train_iq, df_X_val_iq = train_test_split(df_X_train_iq,train_size=0.7,shuffle=False)
    df_y_train_iq = df_X_train_iq['total_cases']
    df_X_train_iq = df_X_train_iq[target_list]
    df_y_val_iq = df_X_val_iq['total_cases']
    df_X_val_iq = df_X_val_iq[target_list]
    
    # if split==False merge datasets about the two cities
    if split == False:
        df_X_train_concat = pd.concat([df_X_train_sj,df_X_train_iq],axis=0, ignore_index=True)
        df_y_train_concat = pd.concat([df_y_train_sj,df_y_train_iq],axis=0, ignore_index=True)
        df_X_val_concat = pd.concat([df_X_val_sj,df_X_val_iq],axis=0, ignore_index=True)
        df_y_val_concat = pd.concat([df_y_val_sj,df_y_val_iq],axis=0, ignore_index=True)
        return df_X_train_concat, df_y_train_concat, df_X_val_concat, df_y_val_concat
    
    else:
        return df_X_train_sj, df_y_train_sj, df_X_val_sj, df_y_val_sj,\
               df_X_train_iq, df_y_train_iq, df_X_val_iq, df_y_val_iq

In [6]:
def get_data_two_weeks_ago(df):
    df_copy = df.copy()
    df_target = df.copy()
    df_copy['two_week_later'] = df_copy['week_start_date']+np.timedelta64(14, 'D')
    df_copy = df_copy[['two_week_later','total_cases']]
    df_target = df_target.merge(df_copy,left_on = 'week_start_date' , right_on = 'two_week_later', suffixes = ('','_twoweeksago'))
    return df_target

In [7]:
feature_list_all = list(X_train.columns)

## 1. 3 weeks ago environmental factors

In [8]:
X_train_sj, y_train_sj, X_val_sj, y_val_sj, X_train_iq, y_train_iq, X_val_iq, y_val_iq = preparing_dataset(X_train, y_train, feature_list_all, 3, split=True) 

In [9]:
X_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [10]:
X_train_sj.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,0,1990,21,1990-05-21,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,0,1990,22,1990-05-28,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,0,1990,23,1990-06-04,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,0,1990,24,1990-06-11,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,0,1990,25,1990-06-18,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [11]:
X_train[X_train['city']=='iq'].head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
936,iq,2000,26,2000-07-01,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,...,43.19,92.418571,25.41,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0
937,iq,2000,27,2000-07-08,0.216833,0.276100,0.289457,0.241657,60.61,296.634286,...,46.00,93.581429,60.61,16.862857,10.314286,26.900000,11.566667,34.0,20.8,55.6
938,iq,2000,28,2000-07-15,0.176757,0.173129,0.204114,0.128014,55.52,296.415714,...,64.77,95.848571,55.52,17.120000,7.385714,26.800000,11.466667,33.0,20.7,38.1
939,iq,2000,29,2000-07-22,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,...,23.96,87.234286,5.60,14.431429,9.114286,25.766667,10.533333,31.5,14.7,30.0
940,iq,2000,30,2000-07-29,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,...,31.80,88.161429,62.76,15.444286,9.500000,26.600000,11.480000,33.3,19.1,4.0


In [12]:
X_train_iq.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,1,2000,29,2000-07-22,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,...,43.19,92.418571,25.41,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0
1,1,2000,30,2000-07-29,0.216833,0.276100,0.289457,0.241657,60.61,296.634286,...,46.00,93.581429,60.61,16.862857,10.314286,26.900000,11.566667,34.0,20.8,55.6
2,1,2000,31,2000-08-05,0.176757,0.173129,0.204114,0.128014,55.52,296.415714,...,64.77,95.848571,55.52,17.120000,7.385714,26.800000,11.466667,33.0,20.7,38.1
3,1,2000,32,2000-08-12,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,...,23.96,87.234286,5.60,14.431429,9.114286,25.766667,10.533333,31.5,14.7,30.0
4,1,2000,33,2000-08-19,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,...,31.80,88.161429,62.76,15.444286,9.500000,26.600000,11.480000,33.3,19.1,4.0


**Create empty dataframes to store results**

In [13]:
train_preds_sj = X_train_sj[['year','weekofyear','week_start_date']]
train_preds_iq = X_train_iq[['year','weekofyear','week_start_date']]

In [14]:
val_preds_sj =  X_val_sj[['year','weekofyear','week_start_date']]
val_preds_iq = X_val_iq[['year','weekofyear','week_start_date']]

**List of selected columns**

In [15]:
target_columns = [ 'ndvi_sw','reanalysis_dew_point_temp_k', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k',
               'station_min_temp_c',  'station_max_temp_c']

In [16]:
X_train_sj[target_columns].head()

,ndvi_sw,reanalysis_dew_point_temp_k,reanalysis_specific_humidity_g_per_kg,reanalysis_min_air_temp_k,station_min_temp_c,station_max_temp_c
0,0.177617,292.414286,14.012857,295.9,20.0,29.4
1,0.155486,293.951429,15.372857,296.4,22.2,31.7
2,0.170843,295.434286,16.848571,297.3,22.8,32.2
3,0.235886,295.310000,16.672857,297.0,23.3,33.3
4,0.247340,295.821429,17.210000,297.5,23.9,35.0


### XGBoost

#### SJ

In [17]:
xgb_regressor = XGBRegressor()
xgb_params = {'max_depth' : (3,4),
              'learning_rate': (0.05,0.06),
              'n_estimators' : (50,55,60,65),
             "min_child_weight" : [ 1, 2, 3],
             "gamma"            : [0.0,0.1, 0.2],
             "colsample_bytree" : [ 0.7,0.8,0.9] }

grid_  = GridSearchCV(xgb_regressor, xgb_params,cv=5, verbose=20, scoring= 'neg_mean_absolute_error') 

# train the model with selected features
grid_.fit(X_train_sj[target_columns],y_train_sj)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-27.164, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-49.091, total=   0.1s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-20.830, total=   0.0s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-33.591, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-28.276, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-27.110, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-49.007, total=   0.0s

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s



[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-21.208, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-33.629, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-28.854, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60 


...

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=65, score=-28.873, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=65, score=-50.125, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=65, score=-21.942, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=65, score=-33.344, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=65 
[CV]  colsample_

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60, score=-28.266, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60, score=-49.064, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60, score=-21.886, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60, score=-33.128, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=60 
[CV]  colsample_

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65, score=-33.743, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65, score=-30.198, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50, score=-29.299, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50, score=-50.369, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50 
[CV]  colsample_

[Parallel(n_jobs=1)]: Done 2160 out of 2160 | elapsed:  2.2min finished
C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constrain...
                                    subsample=None, tree_method=None,
                                    validate_parameters=False, verbosity=None),
             iid='warn', n_jobs=None,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'gamma': [0.0, 0.1, 0.2],
                         'le

In [18]:
grid_.best_estimator_.feature_importances_

array([0.42177373, 0.12136003, 0.11205247, 0.13249585, 0.12325741,
       0.08906054], dtype=float32)

Save feature importance as dataframe

In [21]:
feature_importance = pd.DataFrame(grid_.best_estimator_.feature_importances_, index = target_columns).rename(columns={0:'XGB'})

In-sample Score

In [22]:
y_pred_train = grid_.best_estimator_.predict(X_train_sj[target_columns])
score = mean_absolute_error(y_train_sj, y_pred_train)
print(score)

21.06232852702119


In [23]:
train_preds_sj['XGB'] = y_pred_train

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Out-of-sample Score

In [24]:
y_pred_val = grid_.best_estimator_.predict(X_val_sj[target_columns])

In [25]:
score = mean_absolute_error(y_val_sj, y_pred_val)
print(score)

21.720159891673497


In [26]:
val_preds_sj['XGB'] = y_pred_val

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Iq

In [27]:
xgb_regressor = XGBRegressor()
xgb_params = {'max_depth' : (3,4),
              'learning_rate': (0.05,0.06),
              'n_estimators' : (50,55,60,65),
             "min_child_weight" : [ 1, 2, 3],
             "gamma"            : [0.0,0.1, 0.2],
             "colsample_bytree" : [ 0.7,0.8,0.9] }

grid_  = GridSearchCV(xgb_regressor, xgb_params,cv=5, verbose=20, scoring= 'neg_mean_absolute_error') 
grid_.fit(X_train_iq[target_columns],y_train_iq)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-6.331, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-8.268, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-4.579, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s



[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, score=-3.763, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-6.475, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-8.269, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-4.650, total=   0.0s
[CV] colsample_bytr

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.3s remaining:    0.0s


[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-8.392, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=55, score=-3.842, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60, score=-6.557, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60 
[CV]  colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60, score=-8.277, total=   0.0s
[CV] colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=60 
[CV]  colsample_bytr

...

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=50, score=-6.629, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=50, score=-7.935, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=50, score=-5.189, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=50 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=50, score=-8.353, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=2, n_estimators=50 
[CV]  colsample_bytr

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65, score=-8.002, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65, score=-5.198, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65, score=-8.268, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=3, min_child_weight=3, n_estimators=65, score=-4.129, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=1, n_estimators=50 
[CV]  colsample_bytr

[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=60, score=-8.074, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=60, score=-5.165, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=60, score=-8.584, total=   0.0s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=60 
[CV]  colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=60, score=-4.115, total=   0.1s
[CV] colsample_bytree=0.9, gamma=0.2, learning_rate=0.06, max_depth=4, min_child_weight=2, n_estimators=65 
[CV]  colsample_bytr

[Parallel(n_jobs=1)]: Done 2160 out of 2160 | elapsed:  1.8min finished
C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constrain...
                                    subsample=None, tree_method=None,
                                    validate_parameters=False, verbosity=None),
             iid='warn', n_jobs=None,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'gamma': [0.0, 0.1, 0.2],
                         'le

In-sample Score

In [28]:
y_pred_train = grid_.best_estimator_.predict(X_train_iq[target_columns])
score = mean_absolute_error(y_train_iq, y_pred_train)
print(score)

3.8779626784232186


In [29]:
train_preds_iq['XGB'] = y_pred_train

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Out-of-sample Score

In [30]:
y_pred_val = grid_.best_estimator_.predict(X_val_iq[target_columns])
score = mean_absolute_error(y_val_iq, y_pred_val)
print(score)

7.685811805419433


In [31]:
val_preds_iq['XGB'] = y_pred_val

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Support Vector Machine

Here I decided not to use 'mean absolute error' for optimization (gridsearchcv) for better result

#### SJ

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
param_grid ={'kernel':["rbf"],
            'C':[np.exp(i) for i in np.linspace(-5,5,100)],
            "gamma": np.logspace(-2, 2, 5)}
svr=SVR()
grid_=GridSearchCV(svr, cv=5, param_grid=param_grid, verbose=3)
grid_.fit(X_train_sj[target_columns],y_train_sj)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.395, total=   0.0s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.175, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.047, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.273, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.296, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-0.395, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-0.175, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.00

[CV]  C=0.008246389605637307, gamma=0.1, kernel=rbf, score=-0.047, total=   0.0s
[CV] C=0.008246389605637307, gamma=0.1, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=0.1, kernel=rbf, score=-0.271, total=   0.0s
[CV] C=0.008246389605637307, gamma=0.1, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=0.1, kernel=rbf, score=-0.296, total=   0.0s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-0.395, total=   0.0s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-0.176, total=   0.0s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-0.047, total=   0.0s
[CV] C=0.008246389605637307, gamma=1.0, kernel=rbf ...................
[CV]  C=0.008246389605637307, gamma=1.0, kernel=rbf, score=-0.276, total=   0.0s
[CV] C=

[CV]  C=0.010092531380432784, gamma=1.0, kernel=rbf, score=-0.299, total=   0.0s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-0.395, total=   0.0s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-0.176, total=   0.0s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-0.047, total=   0.1s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-0.277, total=   0.0s
[CV] C=0.010092531380432784, gamma=10.0, kernel=rbf ..................
[CV]  C=0.010092531380432784, gamma=10.0, kernel=rbf, score=-0.304, total=   0.0s
[CV] C=0.010092531380432784, gamma=100.0, kernel=rbf .................
[CV]  C=0.010092531380432784, gamma=100.0, kernel=rbf, score=-0.395, total=   0.1s


...

[CV]  C=66.14962303592083, gamma=100.0, kernel=rbf, score=-0.154, total=   0.1s
[CV] C=66.14962303592083, gamma=100.0, kernel=rbf ....................
[CV]  C=66.14962303592083, gamma=100.0, kernel=rbf, score=-1.110, total=   0.1s
[CV] C=73.18052201955366, gamma=0.01, kernel=rbf .....................
[CV]  C=73.18052201955366, gamma=0.01, kernel=rbf, score=-0.303, total=   0.1s
[CV] C=73.18052201955366, gamma=0.01, kernel=rbf .....................
[CV]  C=73.18052201955366, gamma=0.01, kernel=rbf, score=-0.187, total=   0.1s
[CV] C=73.18052201955366, gamma=0.01, kernel=rbf .....................
[CV]  C=73.18052201955366, gamma=0.01, kernel=rbf, score=-0.133, total=   0.1s
[CV] C=73.18052201955366, gamma=0.01, kernel=rbf .....................
[CV]  C=73.18052201955366, gamma=0.01, kernel=rbf, score=-0.150, total=   0.1s
[CV] C=73.18052201955366, gamma=0.01, kernel=rbf .....................
[CV]  C=73.18052201955366, gamma=0.01, kernel=rbf, score=-0.545, total=   0.1s
[CV] C=73.180522019

[CV]  C=89.56364545447869, gamma=0.01, kernel=rbf, score=-0.153, total=   0.0s
[CV] C=89.56364545447869, gamma=0.01, kernel=rbf .....................
[CV]  C=89.56364545447869, gamma=0.01, kernel=rbf, score=-0.561, total=   0.1s
[CV] C=89.56364545447869, gamma=0.1, kernel=rbf ......................
[CV]  C=89.56364545447869, gamma=0.1, kernel=rbf, score=-0.346, total=   0.0s
[CV] C=89.56364545447869, gamma=0.1, kernel=rbf ......................
[CV]  C=89.56364545447869, gamma=0.1, kernel=rbf, score=-0.204, total=   0.1s
[CV] C=89.56364545447869, gamma=0.1, kernel=rbf ......................
[CV]  C=89.56364545447869, gamma=0.1, kernel=rbf, score=-0.895, total=   0.1s
[CV] C=89.56364545447869, gamma=0.1, kernel=rbf ......................
[CV]  C=89.56364545447869, gamma=0.1, kernel=rbf, score=-0.184, total=   0.1s
[CV] C=89.56364545447869, gamma=0.1, kernel=rbf ......................
[CV]  C=89.56364545447869, gamma=0.1, kernel=rbf, score=-0.859, total=   0.1s
[CV] C=89.56364545447869, 

[CV]  C=109.61450350070197, gamma=0.1, kernel=rbf, score=-0.181, total=   0.1s
[CV] C=109.61450350070197, gamma=0.1, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=0.1, kernel=rbf, score=-0.886, total=   0.1s
[CV] C=109.61450350070197, gamma=1.0, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=1.0, kernel=rbf, score=-0.871, total=   0.2s
[CV] C=109.61450350070197, gamma=1.0, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=1.0, kernel=rbf, score=-0.196, total=   0.1s
[CV] C=109.61450350070197, gamma=1.0, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=1.0, kernel=rbf, score=-2.087, total=   0.1s
[CV] C=109.61450350070197, gamma=1.0, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=1.0, kernel=rbf, score=-0.186, total=   0.1s
[CV] C=109.61450350070197, gamma=1.0, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=1.0, kernel=rbf, score=-2.135, total=   0.1s
[CV] C=109.6145035007

[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  2.3min finished
C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.006737946999085467, 0.0074541086721749295,
                               0.008246389605637307, 0.009122880349438393,
                               0.010092531380432784...
                               0.050803096080009554, 0.05620284622522774,
                               0.06217652402211632, 0.06878513098750348,
                               0.07609615235623413, 0.08418424622139925,
                               0.09313200592177617, 0.10303080346176416,
                               0.11398172257658408, 0.1260965909709399, ...],
                         'gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 

In-sample Score

In [33]:
y_pred_train = grid_.best_estimator_.predict(X_train_sj[target_columns])
score = mean_absolute_error(y_train_sj, y_pred_train)
print(score)

26.99181209676655


In [34]:
train_preds_sj['SVM'] = y_pred_train

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Out-of-sample Score

In [35]:
y_pred_val = grid_.best_estimator_.predict(X_val_sj[target_columns])
score = mean_absolute_error(y_val_sj, y_pred_val)
print(score)

16.608510488931547


In [36]:
val_preds_sj['SVM'] = y_pred_val

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Iq

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
param_grid ={'kernel':["rbf"],
            'C':[np.exp(i) for i in np.linspace(-5,5,100)],
            "gamma": np.logspace(-2, 2, 5)}
svr=SVR()
grid_=GridSearchCV(svr, cv=5, param_grid=param_grid, verbose=3)
grid_.fit(X_train_iq[target_columns],y_train_iq)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-185.445, total=   0.0s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.746, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.115, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.271, total=   0.0s

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] C=0.006737946999085467, gamma=0.01, kernel=rbf ..................
[CV]  C=0.006737946999085467, gamma=0.01, kernel=rbf, score=-0.198, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-182.362, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-0.746, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-0.098, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-0.271, total=   0.0s
[CV] C=0.006737946999085467, gamma=0.1, kernel=rbf ...................
[CV]  C=0.006737946999085467, gamma=0.1, kernel=rbf, score=-0.193, total=   0.0s
[CV] C=0.006737946999085467, gamma=1.0, kernel=rbf ...................
[CV]  C=0.006

...

[CV] C=80.95871990300357, gamma=100.0, kernel=rbf ....................
[CV]  C=80.95871990300357, gamma=100.0, kernel=rbf, score=-1.375, total=   0.0s
[CV] C=80.95871990300357, gamma=100.0, kernel=rbf ....................
[CV]  C=80.95871990300357, gamma=100.0, kernel=rbf, score=-0.157, total=   0.0s
[CV] C=80.95871990300357, gamma=100.0, kernel=rbf ....................
[CV]  C=80.95871990300357, gamma=100.0, kernel=rbf, score=-0.000, total=   0.0s
[CV] C=89.56364545447869, gamma=0.01, kernel=rbf .....................
[CV]  C=89.56364545447869, gamma=0.01, kernel=rbf, score=-154.913, total=   0.0s
[CV] C=89.56364545447869, gamma=0.01, kernel=rbf .....................
[CV]  C=89.56364545447869, gamma=0.01, kernel=rbf, score=-0.754, total=   0.0s
[CV] C=89.56364545447869, gamma=0.01, kernel=rbf .....................
[CV]  C=89.56364545447869, gamma=0.01, kernel=rbf, score=-0.625, total=   0.0s
[CV] C=89.56364545447869, gamma=0.01, kernel=rbf .....................
[CV]  C=89.5636454544786

[CV]  C=109.61450350070197, gamma=0.1, kernel=rbf, score=-257.061, total=   0.0s
[CV] C=109.61450350070197, gamma=0.1, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=0.1, kernel=rbf, score=-0.820, total=   0.0s
[CV] C=109.61450350070197, gamma=0.1, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=0.1, kernel=rbf, score=-1.603, total=   0.0s
[CV] C=109.61450350070197, gamma=0.1, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=0.1, kernel=rbf, score=-0.235, total=   0.0s
[CV] C=109.61450350070197, gamma=0.1, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=0.1, kernel=rbf, score=-0.132, total=   0.0s
[CV] C=109.61450350070197, gamma=1.0, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=1.0, kernel=rbf, score=-239.163, total=   0.1s
[CV] C=109.61450350070197, gamma=1.0, kernel=rbf .....................
[CV]  C=109.61450350070197, gamma=1.0, kernel=rbf, score=-0.679, total=   0.0s
[CV] C=109.614503

[CV]  C=134.15420192797174, gamma=1.0, kernel=rbf, score=-247.703, total=   0.0s
[CV] C=134.15420192797174, gamma=1.0, kernel=rbf .....................
[CV]  C=134.15420192797174, gamma=1.0, kernel=rbf, score=-0.702, total=   0.0s
[CV] C=134.15420192797174, gamma=1.0, kernel=rbf .....................
[CV]  C=134.15420192797174, gamma=1.0, kernel=rbf, score=-5.974, total=   0.0s
[CV] C=134.15420192797174, gamma=1.0, kernel=rbf .....................
[CV]  C=134.15420192797174, gamma=1.0, kernel=rbf, score=-0.135, total=   0.0s
[CV] C=134.15420192797174, gamma=1.0, kernel=rbf .....................
[CV]  C=134.15420192797174, gamma=1.0, kernel=rbf, score=-0.476, total=   0.0s
[CV] C=134.15420192797174, gamma=10.0, kernel=rbf ....................
[CV]  C=134.15420192797174, gamma=10.0, kernel=rbf, score=-366.778, total=   0.0s
[CV] C=134.15420192797174, gamma=10.0, kernel=rbf ....................
[CV]  C=134.15420192797174, gamma=10.0, kernel=rbf, score=-0.278, total=   0.0s
[CV] C=134.1542

[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  1.1min finished
C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.006737946999085467, 0.0074541086721749295,
                               0.008246389605637307, 0.009122880349438393,
                               0.010092531380432784...
                               0.050803096080009554, 0.05620284622522774,
                               0.06217652402211632, 0.06878513098750348,
                               0.07609615235623413, 0.08418424622139925,
                               0.09313200592177617, 0.10303080346176416,
                               0.11398172257658408, 0.1260965909709399, ...],
                         'gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 

In-sample Score

In [38]:
y_pred_train = grid_.best_estimator_.predict(X_train_iq[target_columns])
score = mean_absolute_error(y_train_iq, y_pred_train)
print(score)

4.879340231054772


In [39]:
train_preds_iq['SVM'] = y_pred_train

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Out-of-sample Score

In [40]:
y_pred_val = grid_.best_estimator_.predict(X_val_iq[target_columns])
score = mean_absolute_error(y_val_iq, y_pred_val)
print(score)

7.546303075659086


In [41]:
val_preds_iq['SVM'] = y_pred_val

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Random Forest

#### SJ

In [42]:
from sklearn.ensemble import RandomForestRegressor

rand_ = RandomForestRegressor(bootstrap=True)
rand_params = {    
    'max_depth': (7,8,9,10),
    'n_estimators':(70,80,90,100,200,400,600),
    'min_samples_leaf':(3,4,5)
}

grid_=GridSearchCV(rand_, cv=5, param_grid=rand_params, verbose=3)
grid_.fit(X_train_sj[target_columns],y_train_sj)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=-0.705, total=   0.4s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=-0.118, total=   0.4s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


...

[CV]  max_depth=9, min_samples_leaf=4, n_estimators=200, score=-3.261, total=   1.7s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=200 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=200, score=-0.044, total=   1.2s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=200 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=200, score=-3.379, total=   1.1s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-0.403, total=   2.0s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-0.123, total=   2.1s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-3.380, total=   2.1s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-

[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=-0.456, total=   0.4s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=80 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=-0.121, total=   0.4s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=80 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=-2.950, total=   0.4s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=80 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=-0.035, total=   0.4s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=80 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=-3.398, total=   0.4s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=-0.495, total=   0.4s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=-

[CV]  max_depth=10, min_samples_leaf=4, n_estimators=200, score=-0.043, total=   1.1s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=200 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=200, score=-3.509, total=   1.0s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-0.431, total=   1.9s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-0.123, total=   1.8s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-3.227, total=   1.9s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-0.047, total=   1.9s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, 

[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed:  7.9min finished
C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': (7, 8, 9, 10),
   

In-sample Score

In [43]:
grid_.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [44]:
y_pred_train = grid_.best_estimator_.predict(X_train_sj[target_columns])
score = mean_absolute_error(y_train_sj, y_pred_train)
print(score)

19.675386034404433


In [41]:
train_preds_sj['RF'] = y_pred_train

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


save feature importance of RF and export that as a csv file

In [45]:
feature_importance['RF'] = grid_.best_estimator_.feature_importances_

In [47]:
feature_importance.to_csv('fi_approach1.csv')

Out-of-sample

In [42]:
y_pred_val = grid_.best_estimator_.predict(X_val_sj[target_columns])
score = mean_absolute_error(y_val_sj, y_pred_val)
print(score)

25.88031552890059


In [43]:
val_preds_sj['RF'] = y_pred_val

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Iq

In [44]:
from sklearn.ensemble import RandomForestRegressor

rand_ = RandomForestRegressor(bootstrap=True)
rand_params = {    
    'max_depth': (7,8,9,10),
    'n_estimators':(70,80,90,100,200,400,600),
    'min_samples_leaf':(3,4,5)
}

grid_=GridSearchCV(rand_, cv=5, param_grid=rand_params, verbose=3)
grid_.fit(X_train_iq[target_columns],y_train_iq)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=-354.923, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=-0.731, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=-3.927, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=-0.122, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=70 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=70, score=0.027, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=-369.978, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=-0.668, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=-4.410, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=-0.122, total=   0.1s
[CV] max_depth=7, min_samples_leaf=3, n_estimators=80 ................
[CV]  max_depth=7, min_samples_leaf=3, n_estimators=80, score=-0.067,

...

[CV]  max_depth=9, min_samples_leaf=4, n_estimators=200, score=0.034, total=   0.3s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-372.242, total=   0.6s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-0.697, total=   0.6s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-3.748, total=   0.5s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=-0.111, total=   0.6s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=400 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=400, score=0.021, total=   0.6s
[CV] max_depth=9, min_samples_leaf=4, n_estimators=600 ...............
[CV]  max_depth=9, min_samples_leaf=4, n_estimators=600, score=-

[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=-0.119, total=   0.1s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=80 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=80, score=-0.062, total=   0.1s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=-403.356, total=   0.1s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=-0.670, total=   0.1s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=-4.522, total=   0.1s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score=-0.120, total=   0.1s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=90 ...............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=90, score

[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-0.688, total=   0.6s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-3.705, total=   0.6s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-0.101, total=   0.6s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=400 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=400, score=-0.007, total=   0.6s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=600 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=600, score=-359.980, total=   0.9s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=600 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=600, score=-0.687, total=   0.8s
[CV] max_depth=10, min_samples_leaf=4, n_estimators=600 ..............
[CV]  max_depth=10, min_samples_leaf=4, n_estimators=600

[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed:  2.3min finished
C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': (7, 8, 9, 10),
   

In-sample score

In [45]:
y_pred_train = grid_.best_estimator_.predict(X_train_iq[target_columns])
score = mean_absolute_error(y_train_iq, y_pred_train)
print(score)

4.310226279612345


In [46]:
train_preds_iq['RF'] = y_pred_train

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Out-of-sample score

In [47]:
y_pred_val = grid_.best_estimator_.predict(X_val_iq[target_columns])
score = mean_absolute_error(y_val_iq, y_pred_val)
print(score)

8.002200868929805


In [48]:
val_preds_iq['RF'] = y_pred_val

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


**Export results to csv**

In [49]:
train_preds_sj.to_csv('approach1_insample_prediction_sj.csv',index=False)

In [50]:
val_preds_sj.to_csv('approach1_outsample_prediction_sj.csv',index=False)

In [51]:
train_preds_iq.to_csv('approach1_insample_prediction_iq.csv',index=False)

In [52]:
val_preds_iq.to_csv('approach1_outsample_prediction_iq.csv',index=False)